In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
P = '1'
location = '..\\IHIO_share_A\\prescription\\part' + P + '.pkl'

In [3]:
part = pd.read_pickle(location)

In [4]:
part['date'] = pd.to_datetime(part['date'])

In [5]:
service = pd.read_pickle(r'../IHIO_share_A/service.pkl')
service.set_index('service', inplace=True)

In [6]:
dm_filt = service['atc_3'].isin(['A10A', 'A10B'])
dm_service_codes = service.loc[dm_filt].index
dm_service_codes.shape

(52,)

In [ ]:
part

In [8]:
is_dm_part = part['service'].isin(dm_service_codes)

In [9]:
part_dm = part.loc[part['member'].isin(part.loc[is_dm_part, 'member'].unique())]

In [ ]:
part_dm

In [11]:
# Importing ab_service
ab_service = pd.read_csv('.\\ab_service_final.csv', index_col=0)

In [12]:
ddd = ab_service['calc_ddd']

## Calculating days of ab use for each member.

In [13]:
part_dm['member'].unique().shape

(240528,)

In [14]:
part['member'].unique().shape

(2798343,)

In [15]:
# ab_mask is the services of part_dm that are ab
ab_mask = part_dm['service'].isin(ddd.index)
part_dm_ab = part_dm.loc[ab_mask]
part_dm_ab.loc[:, 'days'] = part_dm_ab['quantity'] * ddd[part_dm_ab['service']].values

C:\Users\a.bagherian\AppData\Local\Temp\ipykernel_6524\3359435875.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part_dm_ab.loc[:, 'days'] = part_dm_ab['quantity'] * ddd[part_dm_ab['service']].values


In [ ]:
part_dm_ab

In [17]:
part_dm_ab['member'].nunique()

196745

In [18]:
ab_mask.sum()

1198457

In [19]:
part_dm_ab['days'].describe()

count    1.198457e+06
mean     8.152347e+00
std      9.851592e+00
min      3.300000e-03
25%      4.000000e+00
50%      6.666000e+00
75%      1.000000e+01
max      5.510250e+03
Name: days, dtype: float64

In [20]:
# Calculating sum of days for each member.
part_dm_ab.groupby('member')['days'].sum().describe()

count    196745.000000
mean         49.659395
std          71.241458
min           0.003300
25%          14.666500
50%          30.167300
75%          60.165600
max        6557.741000
Name: days, dtype: float64

In [21]:
# sns.histplot(x=np.log10(part_dm_ab['days']), bins=20)
# plt.gca().set_title('log10 days for members in part1')
# plt.xticks(np.arange(-2.5, 4, 0.5))

In [22]:
# The sum of days for each dm patient.
# sns.histplot(x=np.log10(part_dm_ab.groupby('member')['days'].sum()), bins=20)

In [23]:
# sns.kdeplot(x=np.log10(part_dm_ab['days']), bw_adjust=2)

In [24]:
# Non-dm patients
part_n = part.loc[~ part['member'].isin(part_dm['member'].unique())]

In [ ]:
part_n

In [26]:
# Calculating the same statisitics for non-dm patients
# ab_mask_n is the services of part_n that are ab
ab_mask_n = part_n['service'].isin(ddd.index)
part_n_ab = part_n.loc[ab_mask_n]
part_n_ab.loc[:, 'days'] = part_n_ab['quantity'] * ddd[part_n_ab['service']].values

C:\Users\a.bagherian\AppData\Local\Temp\ipykernel_6524\2405128522.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part_n_ab.loc[:, 'days'] = part_n_ab['quantity'] * ddd[part_n_ab['service']].values


In [ ]:
part_n_ab

In [28]:
part_n_ab['member'].nunique()

1977377

In [29]:
part_n_ab['days'].describe()

count    7.972790e+06
mean     5.660123e+00
std      8.526797e+00
min      8.000000e-04
25%      6.666000e-01
50%      4.999800e+00
75%      9.999000e+00
max      6.612300e+03
Name: days, dtype: float64

In [30]:
part_n_ab.groupby('member')['days'].sum().describe()

count    1.977377e+06
mean     2.282163e+01
std      3.793342e+01
min      8.000000e-04
25%      5.000000e+00
50%      1.166600e+01
75%      2.799880e+01
max      8.304149e+03
Name: days, dtype: float64

In [31]:
# sns.histplot(x=np.log10(part_n_ab['days']), bins=20)
# plt.gca().set_title('non-dm log10 days for members in part1')
# plt.xticks(np.arange(-2.5, 4, 0.5))

In [32]:
# The sum of days for each non-dm patient.
# sns.histplot(x=np.log10(part_n_ab.groupby('member')['days'].sum()), bins=20)

In [33]:
part['date'].agg([min, max])

min   2014-03-21
max   2017-03-20
Name: date, dtype: datetime64[ns]

In [34]:
fund_dm_weights = part_dm['fund'].value_counts(normalize=True).sort_index()
fund_dm_weights

1     5.192483e-01
2     2.035421e-02
3     1.747432e-01
4     3.391909e-02
6     3.165925e-07
9     2.517248e-01
12    4.748888e-07
13    9.656072e-06
Name: fund, dtype: float64

In [35]:
part_n['fund'].value_counts(normalize=True).sort_index()

1     3.532387e-01
2     2.604431e-02
3     9.575230e-02
4     6.887149e-02
6     2.276317e-07
9     4.560845e-01
10    1.448565e-07
11    7.242825e-07
12    3.724882e-07
13    7.180744e-06
Name: fund, dtype: float64

In [36]:
fund_cat = sorted(part_n['fund'].unique())
fund_weights = pd.Series(index=fund_cat, dtype=float)
fund_weights.loc[fund_dm_weights.index] = fund_dm_weights
fund_weights

1     5.192483e-01
2     2.035421e-02
3     1.747432e-01
4     3.391909e-02
6     3.165925e-07
9     2.517248e-01
10             NaN
11             NaN
12    4.748888e-07
13    9.656072e-06
dtype: float64

In [37]:
test_weights = fund_weights.loc[part_n['fund']]

In [54]:
df = part_n[['fund', 'prescription']]
df['fund'].value_counts(normalize=True)

9     4.560845e-01
1     3.532387e-01
3     9.575230e-02
4     6.887149e-02
2     2.604431e-02
13    7.180744e-06
11    7.242825e-07
12    3.724882e-07
6     2.276317e-07
10    1.448565e-07
Name: fund, dtype: float64

In [ ]:
df['weight'] = test_weights.values
df

In [52]:
sam = df.sample(n=1000000, weights='weight')

In [53]:
sam['fund'].value_counts(normalize=True).sort_index()

1    0.575265
2    0.001710
3    0.053120
4    0.007392
9    0.362513
Name: fund, dtype: float64